## Import libraries


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

## Read and Load dataset from local disk

In [2]:
bread_basket_data = pd.read_csv('../input/affinity-analysis/BreadBasket_DMS.csv')

bread_basket_data.head(10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


FileNotFoundError: [Errno 2] No such file or directory: '../input/affinity-analysis/BreadBasket_DMS.csv'

## Shape of dataset


In [ ]:
bread_basket_data.shape

## Check NONE values

In [ ]:
bread_basket_data.loc[bread_basket_data['Item'] == 'NONE'].count()

## Remove NONE values from dataset

In [ ]:
bread_basket_data = bread_basket_data.drop(
    bread_basket_data.loc[bread_basket_data['Item'] == 'NONE'].index)

## Again check NONE or NAN values

In [ ]:
bread_basket_data.loc[bread_basket_data['Item'] == 'NONE'].count()

## Shape of dataset after removing NONE values

In [ ]:
bread_basket_data.shape

## Check unique values in item set

In [ ]:
bread_basket_data['Item'].unique()

## Calculating Frequency of each item

In [ ]:
basket_items = {}

for item in bread_basket_data['Item']:
    if item in basket_items:
        basket_items[item] = basket_items[item] + 1
    else:
        basket_items[item] = 1
basket_items

In [ ]:
item_names = []
item_frequencies = []

for key, val in basket_items.items():
    item_names.append(key)
    item_frequencies.append(val)

In [ ]:
items_table = pd.DataFrame({'Names': item_names,
                            'Frequencies': item_frequencies})

items_table

## Plotting Bar plot for items vs frequency

In [ ]:
items_table.sort_values('Frequencies', ascending=False).head()

In [ ]:
items_table.plot.bar(y='Frequencies', x='Names', figsize=(12, 8))

In [ ]:
items_table.sort_values('Frequencies', ascending=False).head(20).\
    plot.bar(y='Frequencies', x='Names', figsize=(12, 8))

## Describe dataset

In [ ]:
bread_basket_data['Transaction'].describe()

## Group items with same transaction Id

In [ ]:
bread_basket_data = bread_basket_data.groupby('Transaction').agg(','.join).reset_index()

bread_basket_data = bread_basket_data.drop(['Date', 'Time'], axis=1)
bread_basket_data.head(10)

In [ ]:
bread_basket_data.shape

## Take items sets

In [ ]:
items_data = bread_basket_data['Item']

items_data.head(10)

## Convert into list and split each item in a list

In [ ]:
items_list = [item.split(',') for item in items_data]

items_list_df = pd.DataFrame({'Items': items_list})

items_list_df.head(10)

## Association Rule Mining

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

transencoder = TransactionEncoder()

transencoder_array = transencoder.fit(items_list).transform(items_list)

In [ ]:
encoded_df = pd.DataFrame(transencoder_array, columns=transencoder.columns_)

encoded_df.head(10)

##Support of each item sets

In [ ]:
item_support_df = apriori(encoded_df, min_support=0.01, use_colnames=True)

item_support_df.shape

In [ ]:
item_support_df.sample(10)

## Association rules

In [ ]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(item_support_df,
                          metric='confidence', min_threshold=0.1)

rules.shape

In [ ]:
rules.sample(10)

In [ ]:
rules.sort_values('confidence', ascending=False).head(10)

In [ ]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))

rules.sample(5)

## Select antecedents where item frequency is Sandwich, Bread

In [ ]:
rules[rules['antecedents'] == {'Juice'}]

## Select antecedents where item frequency is Cake, Hot chocolate

In [ ]:
rules[rules['antecedents'] == {'Cake'}]

In [ ]:
rules[rules['antecedents'] == {'Sandwich'}]